In [1]:
import json
import re  
import pandas as pd   
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import RegexpTokenizer
import warnings

In [2]:
warnings.simplefilter(action='ignore')

In [3]:
with open ('bowie_data.json') as f:
    bowie_data = json.load(f)
bowie_data = pd.json_normalize(bowie_data, record_path=['Songs'], meta=['Name', 'Year'], meta_prefix='Album_', record_prefix='Song_')
bowie_data = bowie_data[bowie_data['Song_Lyrics']!=0]

In [4]:
black_list_words = ['Version', 
                    'Mix', 
                    'Remix', 
                    'Mix)', 
                    '(Mix)', 
                    'Remix)', 
                    'Version)', 
                    'Edit)', 
                    '(Live)', 
                    '(Instrumental)', 
                    'Adler', 
                    'Reprise',
                   ]
# Create a regex pattern
pattern = '|'.join(re.escape(word) for word in black_list_words)

# Filter the DataFrame to remove rows where 'text_column' contains any of the words
bowie_data = bowie_data[~bowie_data['Song_Name'].str.contains(pattern, case=False, na=False)]

In [5]:
def clean_lyrics(lyrics):
    # Remove contributor counts followed by song title and "Lyrics"
    lyrics = re.sub(r'\d+\s+Contributors.*?Lyrics', '', lyrics)
    
    # Remove "You Might Also Like" sections
    lyrics = re.sub(r'You might also like[^\n]*', '', lyrics)
    
    # Remove "Embed" at the end
    lyrics = re.sub(r'\d*Embed$', '', lyrics)
    
    # Remove sections like [Verse 1], [Chorus], [Intro], [Outro], etc.
    lyrics = re.sub(r'\[.*?\]', '', lyrics)
    
    # Remove all newline characters
    lyrics = lyrics.replace('\n', ' ')
    
    # Conver to lowercase
    lyrics = lyrics.lower()
    
    # Remove punctuation
    lyrics = re.sub(r'[^\w\s]', ' ', lyrics)
    
    # Remove zero-width spaces
    lyrics = lyrics.replace('\u200b', '')

    # Collapse multiple spaces into a single space
    lyrics = re.sub(r'\s+', ' ', lyrics)
    
    # Strip leading and trailing whitespace
    lyrics = lyrics.strip()
    
    return lyrics

In [6]:
bowie_data['Song_Lyrics'] = bowie_data['Song_Lyrics'].apply(clean_lyrics)

In [7]:
stop_words = stopwords.words('english')
stop_words.extend(['or', 'm', 'ma', 'ours', 'against', 'nor', "it's", 'o', 
                    'wasn', 'hasn', 'my', 'had', 'didn', 'isn', 'did', 'aren', 'those', 'than', 
                    "mustn't", "you've", 'to', 'she', 'having', "haven't", 'into', 't', 'll', 
                    'himself', 'do', "that'll", 'so', 'of', 'on', 'very', 'for', 'out', 'were', 
                    'should', 'they', 'ain', "should've", 'you', "didn't", 'yours', 'was', 'our',
                     'can', 'myself', "shouldn't", 'have', 'up', 'mightn', "you'll", 'any', 
                    'itself', 'hadn', 'him', 'doesn', 'weren', 'y', 'being', "don't", 'them', 
                    'are','and', 'that', 'your', 'yourself', 'their', 'some', 'ourselves', 've', 
                    'doing', 'been', 'shouldn', 'yourselves', "mightn't", 'most', 'because',
                     'few', 'wouldn', "you'd", 'through', "you're", 'themselves', 'an', 'if',
                     "wouldn't", 'its', 'other', "won't", "wasn't", "she's", 'we', 'shan',
                     "weren't",'don',"hadn't", 'this', 'off', 'while', 'a', 'haven', 'her', 
                    'theirs', 'all', "hasn't", "doesn't", 'about', 'then', 'by','such', 'but', 
                    'until', 'each', 'there', "aren't", 'with', 'not', "shan't", 'hers', 'it', 
                    'too', 'i', 'at', 'is', 'as', 'me', 'herself', 's', 'the', 'where', 'am', 
                    'has', 'over', "couldn't", 'when', 'does', 'mustn','re', 'no', 'in', 'who', 
                    'd', 'own', 'he', 'be', "isn't", 'his', 'these', 'same', 'whom', 'will', 
                    'needn','couldn', 'from', 'ah', 'yo', 'oh', 'yeah', 'ya', 'hey'
                  ]) 

In [8]:
def preprocess_text(text):
    # Tokenise words & ignore punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    
    processed_words = [word for word in tokens if not word in stop_words]
    processed_words = ' '.join(processed_words)
    return processed_words

In [9]:
bowie_data['Song_Lyrics'] = bowie_data['Song_Lyrics'].apply(preprocess_text)
bowie_data.reset_index(drop=True, inplace=True)

In [10]:
album_years = []
album_names = []
song_names = []
words = []

for index, row in bowie_data.iterrows():
    song_words = row['Song_Lyrics'].split()
    
    for word in song_words:
        album_years.append(row['Album_Year'])
        album_names.append(row['Album_Name'])
        song_names.append(row['Song_Name'])
        words.append(word)

lyrics_by_words = pd.DataFrame({
    'Album_Year': album_years,
    'Album_Name': album_names,
    'Song_Name': song_names,
    'Word': words
})

In [11]:
filename = 'bowie_cleaned_data.csv'
bowie_data.to_csv(filename, index=False)

filename = 'lyrics_by_words.csv'
lyrics_by_words.to_csv(filename, index=False)